In [12]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import pandas as pd
import os

# Carregar base de dados
diretorio_atual = os.getcwd()

caminho_dataset = os.path.join(diretorio_atual, 'datasets', 'parkinsons.data')

df = pd.read_csv(caminho_dataset)

X = df.drop(['status', 'name', 'APQ', 'D2',
            'Fhi(Hz)', 'Flo(Hz)', 'Fo(Hz)',
             'PPQ', 'RAP', 'spread1', 'spread2'], axis=1)

y = df['status']

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2)




  Classificador: XGBClassifier
Melhores parâmetros: {'learning_rate': 0.1, 'n_estimators': 50}
Melhor pontuação: 0.910483870967742
Acurácia no conjunto de teste: 0.8974358974358975

  O melhor estimator foi: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


'\napós diversos testes, \né possível concluir que:\nGradientBoostingClassifier(learning_rate=0.1, n_estimators=50)\nfoi o que obteve maior pontuação e acurácia nos testes.\n '

In [52]:
from sklearn.metrics import roc_auc_score

def treinar_modelo(params):
    learning_rate = params[0]
    min_child_weight = params[1]
    max_depth = params[2]
    colsample_bytree = params[3]
    gamma = params[4]
    scale_pos_weight = params[5]
    
    print(params, '\n')

    modelo = XGBClassifier(learning_rate=learning_rate, min_child_weight=min_child_weight,
                            max_depth=max_depth, colsample_bytree=colsample_bytree,
                            gamma=gamma, scale_pos_weight=scale_pos_weight, n_estimators=50) # número de arvores é definido como fixo
    modelo.fit(X_treino, y_treino)

    proba = modelo.predict_proba(X_teste)[:, 1]

    return -1 * roc_auc_score(y_teste, proba) # multiplicado por -1 porque é preciso minimizar a negativa do auc não o próprio auc


space = [(1e-3, 1, 'log-uniform'), # learning_rate, log-uniform dá mais importância para números menores 
         (1, 10), # min_child_weight
         (3, 10), # max_depth
         (0.5, 1.0), # colsample_bytree
         (0, 5), # gamma
         (1, 10)] # scale_pos_weight

# resultado = dummy_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=30)
# resultado.x

Otimização Bayesiana

In [61]:
from skopt import gp_minimize

resultados_gp = gp_minimize(treinar_modelo, space, verbose=1, n_calls=100, n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.
[0.2267600555270784, 4, 7, 0.9869774089375563, 4, 5] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.7436
Function value obtained: -0.8293
Current minimum: -0.8293
Iteration No: 2 started. Evaluating function at random point.
[0.21477724223018627, 1, 8, 0.7415016381244663, 2, 1] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0277
Function value obtained: -0.8534
Current minimum: -0.8534
Iteration No: 3 started. Evaluating function at random point.
[0.008385109936114419, 6, 7, 0.8797853749119202, 4, 3] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0418
Function value obtained: -0.8138
Current minimum: -0.8534
Iteration No: 4 started. Evaluating function at random point.
[0.06733737083449341, 2, 7, 0.5778779662927062, 2, 3] 

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.1203
Function value obtained: -0.8483
Current minimum: -0.8

Exploration Exploitation Tradeoff

In [62]:
resultados_gp.x


[0.001, 1, 10, 0.5361442643156793, 0, 4]